In [ ]:
import os

import pandas as pd

from utils.optuna_database_manager import OptunaDBManager

db_root_path = "../../data/backtesting/"
db_name = "backtesting_report.db"

optuna_db_manager = OptunaDBManager(db_name=db_name,
                                    db_root_path=db_root_path)

In [ ]:
optuna_db_manager.studies

In [ ]:
study_name = "super_trend_optimization_1"
df = optuna_db_manager.merged_df[optuna_db_manager.merged_df["study_name"] == study_name]

In [ ]:
df

### Bar Chart of Average Trading Time
This bar chart compares the average trading time across trials. It helps to quickly identify trials with unusually long or short trading times.


In [ ]:
import plotly.express as px

fig = px.bar(df, x='trial_id', y='avg_trading_time_in_hours', title='Average Trading Time per Trial')
fig.show()

### Scatter Plot of Accuracy vs. Average Trading Time
This scatter plot shows the relationship between the accuracy and the average trading time of each trial. It can help to identify if there is any correlation between these two metrics.


In [ ]:
fig = px.scatter(df, x='avg_trading_time_in_hours', y='accuracy', title='Accuracy vs. Average Trading Time')
fig.show()


### Histogram of Total Positions
The histogram represents the distribution of total positions across all trials. This visualization is useful for understanding the general spread and most common values of positions.


In [ ]:
fig = px.histogram(df, x='total_positions', title='Distribution of Total Positions')
fig.show()

### Pie Chart of Win Signals
This pie chart shows the proportion of win signals in each trial, providing a visual representation of the success rate distribution across trials.


In [ ]:
fig = px.pie(df, names='trial_id', values='win_signals', title='Proportion of Win Signals per Trial')
fig.show()


### Box Plot for Trial Value
A box plot for trial values to identify the range, median, and any potential outliers in the trial values.


In [ ]:
fig = px.box(df, y='value', title='Box Plot of Trial PNL')
fig.show()

### Heatmap for Correlation Analysis
This heatmap illustrates the correlation between various numerical variables such as accuracy, average trading time, total positions, win signals, and value.


In [ ]:
import plotly.express as px

# Calculate the correlation matrix
correlation_matrix = df[['accuracy', 'avg_trading_time_in_hours', 'total_positions', 'win_signals', 'value']].corr()

# Generate the heatmap
fig = px.imshow(correlation_matrix,
                x=correlation_matrix.columns,
                y=correlation_matrix.columns,
                title='Correlation Heatmap',
                labels=dict(x="Variable", y="Variable", color="Correlation"),
                color_continuous_scale='RdBu')

fig.show()



### Stacked Bar Chart for Win Signals vs Total Positions
A stacked bar chart displaying the ratio of win signals to total positions for each trial. This helps in visualizing the efficiency and effectiveness of each trial.


In [ ]:
df['loss_signals'] = df['total_positions'] - df['win_signals']
fig = px.bar(df, x='trial_id', y=['win_signals', 'loss_signals'], title='Win vs Loss Signals per Trial', labels={'value':'Number of Signals'}, hover_data=['total_positions'])
fig.show()


### Scatter Plots Against PNL
These scatter plots show how various metrics behave in relation to the PNL (Profit and Loss). This analysis can help in understanding which factors have a stronger relationship with financial outcomes.


In [ ]:
# Scatter Plot for Accuracy vs PNL
fig_accuracy_pnl = px.scatter(df, x='accuracy', y='value', title='Accuracy vs PNL')
fig_accuracy_pnl.show()

# Scatter Plot for Average Trading Time vs PNL
fig_tradingtime_pnl = px.scatter(df, x='avg_trading_time_in_hours', y='value', title='Average Trading Time vs PNL')
fig_tradingtime_pnl.show()

# Scatter Plot for Total Positions vs PNL
fig_positions_pnl = px.scatter(df, x='total_positions', y='value', title='Total Positions vs PNL')
fig_positions_pnl.show()

# Scatter Plot for Win Signals vs PNL
fig_winsignals_pnl = px.scatter(df, x='win_signals', y='value', title='Win Signals vs PNL')
fig_winsignals_pnl.show()


### Scatter Matrix of All Variables
A scatter matrix allows us to see both the distribution of single variables and the relationships between two variables. The diagonal shows histograms for each variable, and the scatter plots show correlations between them.


In [ ]:
import plotly.express as px

# Selecting columns for scatter matrix
selected_columns = ['accuracy', 'avg_trading_time_in_hours', 'total_positions', 'win_signals', 'value']

# Creating scatter matrix
fig_matrix = px.scatter_matrix(df[selected_columns], dimensions=selected_columns, title='Scatter Matrix of Variables', height=800)
fig_matrix.show()


In [ ]:
import json
from hummingbot.core.data_type.common import PositionMode
from hummingbot.connector.connector_base import TradeType, OrderType
from hummingbot.smart_components.utils.config_encoder_decoder import ConfigEncoderDecoder

trial_to_analyze = 36

trial = df[df["trial_id"] == trial_to_analyze]

In [ ]:
trial

In [ ]:
# Transform trial config in a dictionary
encoder_decoder = ConfigEncoderDecoder(TradeType, OrderType, PositionMode)
trial_config = encoder_decoder.decode(json.loads(trial["config"].item()))
trial_config

In [ ]:
from quants_lab.controllers.supertrend import SuperTrend, SuperTrendConfig

# In this case we are using the supertrend controller but we can also access to the controller by using the method load_controllers
config = SuperTrendConfig(**trial_config)
controller = SuperTrend(config)

In [ ]:
from quants_lab.strategy.strategy_analysis import StrategyAnalysis
from hummingbot.smart_components.strategy_frameworks.directional_trading import DirectionalTradingBacktestingEngine

# Backtest configuration
trade_cost = 0.0006
initial_portfolio_usd = 1000
start = "2023-01-01"
end = "2024-01-02"

# Load the data
engine = DirectionalTradingBacktestingEngine(controller=controller)
engine.load_controller_data("../../data/candles")
backtesting_results = engine.run_backtesting(initial_portfolio_usd=initial_portfolio_usd,
                                             trade_cost=trade_cost,
                                             start=start, end=end)
strategy_analysis = StrategyAnalysis(
    positions=backtesting_results["executors_df"],
    candles_df=backtesting_results["processed_data"],
)

In [ ]:
strategy_analysis.create_base_figure(volume=False, positions=False, trade_pnl=True)
strategy_analysis.figure()

In [ ]:
strategy_analysis.pnl_over_time()